In [1]:
# Login
!az login > /dev/null

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DMBU3DA7F to authenticate.


In [2]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [19]:
# Uploading a dataset
from azureml.core import Dataset
import pandas as pd

# Register a dataset
tab_ds = Dataset.Tabular.from_delimited_files(path=url)
tab_ds = tab_ds.register(workspace=ws, name='diabetes')

In [21]:
# Retrieve DS
retrieved_ds = Dataset.get_by_name(workspace=ws, name='diabetes')
retrieved_ds.to_pandas_dataframe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [27]:
# Run an experiment script on that data
from azureml.core import Experiment, ScriptRunConfig, Environment

# Setup Docker environment
myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

# Create a script config
script_config = ScriptRunConfig(source_directory='scripts',
                                script='diabetes_lr_ds.py',
                                arguments = ['--ds-name', 'diabetes'],
                                compute_target=ws.compute_targets['target'],
                                environment=myenv)

# Submit the experiment
experiment = Experiment(workspace=ws, name='diabetes_lr')
run = experiment.submit(config=script_config)

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None
